In [1]:
import numpy as np
import os
import tensorflow as tf

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

Train a deep MLP on the MNIST dataset and see if you can get over 98% precision. Try adding all the bells and whistles (i.e., save checkpoints, restore the last checkpoint in case of an interruption, add summaries, plot learning curves using TensorBoard, and so on).

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
X_train = mnist.train.images
X_test = mnist.test.images
y_train = mnist.train.labels.astype("int")
y_test = mnist.test.labels.astype("int")

In [4]:
print(X_train.shape, X_test.shape)

(55000, 784) (10000, 784)


In [5]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [6]:
reset_graph()
X = tf.placeholder(dtype=tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(dtype=tf.int64, shape = (None), name="y")

In [7]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(inputs=X, units=n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = tf.layers.dense(inputs=hidden1, units=n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = tf.layers.dense(inputs=hidden2, units=n_outputs, name="outputs")

In [8]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    loss_summary = tf.summary.scalar('log_loss', loss)

In [9]:
learning_rate = 0.01
with tf.name_scope("training"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    trainining_op = optimizer.minimize(loss)

In [10]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(k=1, predictions=logits, targets=y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary = tf.summary.scalar("accuracy", accuracy)

In [11]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [12]:
# define the directory to write TensorBoard logs
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs_11"
logdir = "{0}/run-{1}".format(root_logdir, now)

In [13]:
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [14]:
X_valid = mnist.validation.images
y_valid = mnist.validation.labels

In [30]:
checkpoint_path = "./model/my_deep_mnist_model.ckpt"
final_model_path = "./model/my_deep_mnist_model_final"
checkpoint_epoch_path = checkpoint_path + ".epoch"

batch_size = 500
n_epoches = 101
best_loss = np.infty
epochs_without_progress = 0
max_epochs_without_progress = 50

with tf.Session() as sess:
    if os.path.isfile(checkpoint_epoch_path):
        with open(checkpoint_epoch_path, "rb") as f:
            start_epoch = int(f.read())
        print("Training was interrupted. Continuing at epoch", start_epoch)
        saver.restore(sess, checkpoint_path)
    else:
        start_epoch = 0
        init.run()

    for epoch in range(start_epoch, n_epoches):
        for iteration in range(mnist.train.num_examples//batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(trainining_op, feed_dict={X:X_batch, y:y_batch})
            accuracy_val, loss_val, accuracy_summary_str, loss_summary_str = sess.run([accuracy, loss, accuracy_summary, loss_summary], feed_dict={X: X_valid, y: y_valid})
        file_writer.add_summary(accuracy_summary_str, epoch)
        file_writer.add_summary(loss_summary_str, epoch)
        if epoch%5==0:
            print("Epoch:", epoch,
                  "\tValidation accuracy: {:.3f}%".format(accuracy_val * 100),
                  "\tLoss: {:.5f}".format(loss_val))
            saver.save(sess, checkpoint_path) # checkpoint saver
            with open(checkpoint_epoch_path, "wb") as f:
                f.write(b"%d" %(epoch + 1))
            if loss_val < best_loss:
                best_loss = loss_val
                saver.save(sess, final_model_path) # final model saver
            else:
                epochs_without_progress += 5
                if epochs_without_progress > max_epochs_without_progress:
                    print("Early Stopping")
                    break
                    
os.remove(checkpoint_epoch_path)

Epoch: 0 	Validation accuracy: 69.360% 	Loss: 1.65358
Epoch: 5 	Validation accuracy: 88.380% 	Loss: 0.44611
Epoch: 10 	Validation accuracy: 90.940% 	Loss: 0.33753
Epoch: 15 	Validation accuracy: 91.860% 	Loss: 0.29316
Epoch: 20 	Validation accuracy: 92.900% 	Loss: 0.26612
Epoch: 25 	Validation accuracy: 93.500% 	Loss: 0.24625
Epoch: 30 	Validation accuracy: 93.960% 	Loss: 0.23047
Epoch: 35 	Validation accuracy: 94.060% 	Loss: 0.21773
Epoch: 40 	Validation accuracy: 94.320% 	Loss: 0.20600
Epoch: 45 	Validation accuracy: 94.520% 	Loss: 0.19613
Epoch: 50 	Validation accuracy: 94.740% 	Loss: 0.18755
Epoch: 55 	Validation accuracy: 95.020% 	Loss: 0.17949
Epoch: 60 	Validation accuracy: 95.240% 	Loss: 0.17205
Epoch: 65 	Validation accuracy: 95.520% 	Loss: 0.16581
Epoch: 70 	Validation accuracy: 95.580% 	Loss: 0.15920
Epoch: 75 	Validation accuracy: 95.940% 	Loss: 0.15379
Epoch: 80 	Validation accuracy: 96.000% 	Loss: 0.14900
Epoch: 85 	Validation accuracy: 96.200% 	Loss: 0.14384
Epoch: 90 	V

In [33]:
with tf.Session() as sess:
    saver.restore(sess, final_model_path)
    accuracy_val_test = accuracy.eval(feed_dict={X:X_test, y:y_test})
    accuracy_val_train = accuracy.eval(feed_dict={X:X_train, y:y_train})

INFO:tensorflow:Restoring parameters from ./model/my_deep_mnist_model_final


In [37]:
print("Training Accuracy:{0:.2f}, Test Accuracy:{1:.2f}".format(accuracy_val_train, accuracy_val_test))

Training Accuracy:0.96, Test Accuracy:0.96
